<a href="https://colab.research.google.com/github/TynK-M/nursing-fall-risk-assessment/blob/main/nursing_fall_risk_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Morse Fall Scale

## Risk Categories

In [ ]:
from enum import Enum

In [ ]:
class MorseFallRisk(Enum):
  LOW = 1
  MEDIUM = 2
  HIGH = 3

  def contains(self, score: int) -> bool:
    if self is MorseFallRisk.LOW:
      return score < 25
    elif self is MorseFallRisk.MEDIUM:
      return 25 <= score <= 45
    elif self is MorseFallRisk.HIGH:
      return score > 45
    else:
      raise NotImplementedError(f"No rule for {self}")

  @classmethod
  def from_score(cls, score: int) -> "MorseFallRisk":
    for tier in cls:
      if tier.contains(score):
        return tier.name
    raise ValueError(f"Invalid Morse Fall Scale score: {score}")

## Ambulatory Aid

In [ ]:
low_ambulatory_aid = [
    "bed rest",
    "nurse assist",
]

medium_ambulatory_aid = [
    "crutches",
    "cane",
    "walker",
]

high_ambulatory_aid = [
    "furniture",
]

## Gait or Transfering

In [ ]:
low_gait_or_transfering = [
    "normal",
    "bed rest",
    "immobile",
]

medium_gait_or_transfering = [
    "weak",
]

high_gait_or_transfering = [
    "impaired",
]

## Morse Fall Scale

In [ ]:
def morse_fall_scale(falling_history: bool, secondary_diagnosis: bool, ambulatory_aid: str, iv_or_heparin_lock: bool, gait_or_transfering: str, does_forget_limitations: bool) -> int:
    score = 0

    score += 25 if falling_history else 0
    score += 15 if secondary_diagnosis else 0

    if ambulatory_aid in high_ambulatory_aid:
        score += 30
    elif ambulatory_aid in medium_ambulatory_aid:
        score += 15

    score += 20 if iv_or_heparin_lock else 0

    if gait_or_transfering in high_gait_or_transfering:
        score += 20
    elif gait_or_transfering in medium_gait_or_transfering:
        score += 10

    score += 15 if does_forget_limitations else 0

    return score

# Dataset

The dataset is generated and does not use real patient data.

In [ ]:
import pandas as pd
import random

In [ ]:
NUMBER_OF_SIMULATED_PATIENTS = 200

In [ ]:
def generate_patient() -> dict:
  return {
      "falling_history": random.choice([True, False]),
      "secondary_diagnosis": random.choice([True, False]),
      "ambulatory_aid": random.choice([None] + low_ambulatory_aid + medium_ambulatory_aid + high_ambulatory_aid),
      "iv_or_heparin_lock": random.choice([True, False]),
      "gait_or_transfering": random.choice([None] + low_gait_or_transfering + medium_gait_or_transfering + high_gait_or_transfering),
      "does_forget_limitations": random.choice([True, False])
  }

In [ ]:
data = [generate_patient() for _ in range(NUMBER_OF_SIMULATED_PATIENTS)]

In [ ]:
df = pd.DataFrame(data)

# Output

In [ ]:
df["morse_score"] = df.apply(
    lambda row: morse_fall_scale(
        row["falling_history"],
        row["secondary_diagnosis"],
        row["ambulatory_aid"],
        row["iv_or_heparin_lock"],
        row["gait_or_transfering"],
        row["does_forget_limitations"]
    ),
    axis=1
)

df["risk_level"] = df["morse_score"].apply(MorseFallRisk.from_score)

In [ ]:
df